# Overview
This codelab will demonstrate how to build a LSTM model for MNIST recognition using keras & how to convert the model to TensorFlow Lite.

---



In [ ]:
!pip install tf-nightly

### Prerequisites
We're going to override the environment variable `TF_ENABLE_CONTROL_FLOW_V2` since for TensorFlow Lite control flows.

In [10]:
# This is important!
import os
os.environ['TF_ENABLE_CONTROL_FLOW_V2'] = '1'

import numpy as np

import json
import numpy
import pandas as pd
import tensorflow as tf
from keras import losses
from keras import optimizers
from keras.callbacks import Callback
from keras.layers import Dense
from keras.layers import LSTM, Bidirectional, BatchNormalization, SimpleRNN
from keras.layers import Embedding
from keras.models import Sequential
from keras.preprocessing.sequence import pad_sequences
from keras.preprocessing.text import Tokenizer
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import nltk


In [11]:
import ssl
ssl._create_default_https_context = ssl._create_unverified_context

stop_words = set(stopwords.words('english'))
nltk.download('wordnet')

from nltk.stem import WordNetLemmatizer
import re

import matplotlib.pyplot as plt

numpy.random.seed(7)

[nltk_data] Downloading package wordnet to /Users/bsuthar/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [12]:
# GLOVE--EMBEDDING
def read_data(file_name):
    with open(file_name,'r') as f:
        word_vocab = set() # not using list to avoid duplicate entry
        word2vector = {}
        for line in f:
            line_ = line.strip() #Remove white space
            words_Vec = line_.split()
            word_vocab.add(words_Vec[0])
            word2vector[words_Vec[0]] = numpy.array(words_Vec[1:],dtype=float)
    print("Total Words in DataSet:",len(word_vocab))
    return word_vocab,word2vector

word_vocab,w2v = read_data('Glove/glove.twitter.27B.25d.txt')

Total Words in DataSet: 1193514


In [13]:
## While preprocessing total word count in corpus is stored
word_count = []


def preprocess(text):
    text = str(text.lower())

    # Remove all the special characters
    text = re.sub(r'\W', ' ', text)

    text = re.sub(r'[^a-zA-Z ]+', '', text)

    # remove all single characters
    text = re.sub(r'\s+[a-zA-Z]\s+', ' ', text)

    # Remove single characters from the start
    text = re.sub(r'\^[a-zA-Z]\s+', ' ', text)

    # Substituting multiple spaces with single space
    text = re.sub(r'\s+', ' ', text, flags=re.I)

    # Removing prefixed 'b'
    text = re.sub(r'^b\s+', '', text)

    text = word_tokenize(text)

    text = [word for word in text if word not in stop_words]



    word_net_lemmatizer = WordNetLemmatizer()

    text = [word_net_lemmatizer.lemmatize(word) for word in text]

    # ps = nltk.stem.PorterStemmer()
    #
    # text = [ps.stem(word) for word in text]

    #word_count.append(len(text))

    text = ' '.join(text)

    return text



In [34]:
df = pd.read_excel('Dset/drug_names.xlsx')

emgality = df['emgality'].values
olumiant = df['olumiant'].values
trulicity = df['trulicity'].values


appeneded = emgality + olumiant + trulicity
feature_emgality = []
feature_olumiant = []
feature_trulicity = []



for i in range(len(emgality)):
    feature_emgality.append(0)
for i in range(len(olumiant)):
    feature_olumiant.append(1)
for i in range(len(trulicity)):
    feature_trulicity.append(2)

    
data_emgality = pd.DataFrame({'text':emgality, 'feature':feature_emgality})
data_olumiant = pd.DataFrame({'text':olumiant, 'feature':feature_olumiant} )
data_trulicity = pd.DataFrame({'text':trulicity, 'feature':feature_trulicity} )


data_frame = pd.concat([data_emgality,data_olumiant,data_trulicity])
data_frame

,text,feature
0,m gallery,0
1,in gallatin,0
2,m garrity,0
3,am gallatin,0
4,an reality,0
5,in reality,0
6,and garity,0
7,am galtee,0
8,adam reality,0
9,and galatea,0


In [35]:
data_frame.text = data_frame.text.apply(preprocess)

X = data_frame.text
y = data_frame.feature

In [36]:
word_count = numpy.array(word_count)
max_length = 10


tokenizer = Tokenizer()
tokenizer.fit_on_texts(data_frame.text)

X = tokenizer.texts_to_sequences(X)
X = pad_sequences(X, maxlen=max_length, padding='post')

num_words = len(tokenizer.word_index) + 1
embedding_matrix = numpy.zeros((num_words, 25))
embedding_matrix.shape

(28, 25)

In [37]:
dummy_matrix = numpy.zeros(shape = (25,))

for word,i in tokenizer.word_index.items():
    try:

        embedding_vector = w2v[word]
        if embedding_vector is not None:
            print(i,word)
            embedding_matrix[i] = embedding_vector
    except:
        print('dummy matrix')
        #embedding_matrix[i] = dummy_matrix
        
    

embedding_matrix.shape

1 city
2 reality
3 truly
4 gallatin
5 gallery
6 garrity
dummy matrix
dummy matrix
9 adam
10 galatea
11 event
12 lament
13 element
14 ornament
15 limit
16 aluminium
17 lumen
18 idiot
dummy matrix
20 lumiere
21 sweetie
22 sweety
23 julie
24 holy
25 rowley
26 list
27 luxury


(28, 25)

## Step 1 Build the MNIST LSTM model.

Note we will be using **`tf.lite.experimental.nn.TFLiteLSTMCell`** & **`tf.lite.experimental.nn.dynamic_rnn`** in the tutorial.

Also note here, we're not trying to build the model to be a real world application, but only demonstrates how to use TensorFlow lite. You can a build a much better model using CNN models.

For more canonical lstm codelab, please see [here](https://github.com/keras-team/keras/blob/master/examples/imdb_lstm.py).

In [38]:
# Step 1: Build the MNIST LSTM model.
def buildLstmLayer(inputs, num_layers, num_units):
  """Build the lstm layer.

  Args:
    inputs: The input data.
    num_layers: How many LSTM layers do we want.
    num_units: The unmber of hidden units in the LSTM cell.
  """
  lstm_cells = []
  for i in range(num_layers):
    lstm_cells.append(
        tf.lite.experimental.nn.TFLiteLSTMCell(
            num_units, forget_bias=0, name='rnn{}'.format(i)))
  lstm_layers = tf.keras.layers.StackedRNNCells(lstm_cells)
  # Assume the input is sized as [batch, time, input_size], then we're going
  # to transpose to be time-majored.
  transposed_inputs = tf.transpose(
      inputs, perm=[1, 0, 2])
  outputs, _ = tf.lite.experimental.nn.dynamic_rnn(
      lstm_layers,
      transposed_inputs,
      dtype='float32',
      time_major=True)
  unstacked_outputs = tf.unstack(outputs, axis=0)
  return unstacked_outputs[-1]

tf.reset_default_graph()
model = tf.keras.models.Sequential([
  tf.keras.layers.Input(shape=(X.shape[1],), name='input'),
  tf.keras.layers.Embedding(num_words , 25,weights=[embedding_matrix],trainable=True,input_length=X.shape[1]),
  tf.keras.layers.Lambda(buildLstmLayer, arguments={'num_layers' : 2, 'num_units' : 64}),
  tf.keras.layers.Flatten(),
  tf.keras.layers.Dense(4, activation=tf.nn.softmax, name='output')
])
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 10, 25)            700       
_________________________________________________________________
lambda (Lambda)              (None, 64)                56064     
_________________________________________________________________
flatten (Flatten)            (None, 64)                0         
_________________________________________________________________
output (Dense)               (None, 4)                 260       
Total params: 57,024
Trainable params: 57,024
Non-trainable params: 0
_________________________________________________________________


In [24]:
import tensorflow as tf
tf.__version__

'1.14.0'

## Step 2: Train & Evaluate the model.
We will train the model using MNIST data.

In [44]:
# Step 2: Train & Evaluate the model.
from sklearn.model_selection import train_test_split
x_train, x_test,y_train, y_test = train_test_split(X,y)

# x_train, x_test = x_train / 255.0, x_test / 255.0

# # Cast x_train & x_test to float32.
# x_train = x_train.astype(np.float32)
# x_test = x_test.astype(np.float32)

model.fit(X, y, epochs=100,validation_data=(x_test,y_test))
# model.evaluate(x_test, y_test)


Train on 30 samples, validate on 8 samples
Epoch 1/100
30/30 [==============================] - 0s 716us/sample - loss: 0.3540 - acc: 0.8333 - val_loss: 8.8891e-04 - val_acc: 1.0000
Epoch 2/100
30/30 [==============================] - 0s 641us/sample - loss: 0.1698 - acc: 0.9667 - val_loss: 9.1544e-04 - val_acc: 1.0000
Epoch 3/100
30/30 [==============================] - 0s 616us/sample - loss: 0.1557 - acc: 0.9667 - val_loss: 9.3533e-04 - val_acc: 1.0000
Epoch 4/100
30/30 [==============================] - 0s 679us/sample - loss: 0.1840 - acc: 0.9000 - val_loss: 9.9561e-04 - val_acc: 1.0000
Epoch 5/100
30/30 [==============================] - 0s 612us/sample - loss: 0.1126 - acc: 0.9667 - val_loss: 0.0011 - val_acc: 1.0000
Epoch 6/100
30/30 [==============================] - 0s 624us/sample - loss: 0.0734 - acc: 0.9667 - val_loss: 0.0011 - val_acc: 1.0000
Epoch 7/100
30/30 [==============================] - 0s 634us/sample - loss: 0.0448 - acc: 0.9667 - val_loss: 0.0012 - val_acc: 1.0

Epoch 60/100
30/30 [==============================] - 0s 643us/sample - loss: 0.0012 - acc: 1.0000 - val_loss: 6.0596e-04 - val_acc: 1.0000
Epoch 61/100
30/30 [==============================] - 0s 591us/sample - loss: 0.0012 - acc: 1.0000 - val_loss: 6.0318e-04 - val_acc: 1.0000
Epoch 62/100
30/30 [==============================] - 0s 564us/sample - loss: 0.0012 - acc: 1.0000 - val_loss: 6.0051e-04 - val_acc: 1.0000
Epoch 63/100
30/30 [==============================] - 0s 636us/sample - loss: 0.0012 - acc: 1.0000 - val_loss: 5.9787e-04 - val_acc: 1.0000
Epoch 64/100
30/30 [==============================] - 0s 581us/sample - loss: 0.0012 - acc: 1.0000 - val_loss: 5.9525e-04 - val_acc: 1.0000
Epoch 65/100
30/30 [==============================] - 0s 552us/sample - loss: 0.0012 - acc: 1.0000 - val_loss: 5.9259e-04 - val_acc: 1.0000
Epoch 66/100
30/30 [==============================] - 0s 579us/sample - loss: 0.0011 - acc: 1.0000 - val_loss: 5.9004e-04 - val_acc: 1.0000
Epoch 67/100
30/30 [

In [40]:
x_train.shape,y_train.shape

((22, 10), (22,))

In [45]:
data = 'truly city'
data = preprocess(data)
data = tokenizer.texts_to_sequences([data])
data = pad_sequences(data, maxlen=max_length, padding='post')
results = model.predict([data])
np.argmax(results)

2

## Step 3: Convert the Keras model to TensorFlow Lite model.

Note here: we just convert to TensorFlow Lite model as usual.

In [78]:
idx = 6
y_train

8     1
38    3
12    0
35    0
0     1
     ..
21    1
31    3
27    1
6     3
11    0
Name: feature, Length: 117, dtype: int64

In [46]:
# Step 3: Convert the Keras model to TensorFlow Lite model.
sess = tf.keras.backend.get_session()
input_tensor = sess.graph.get_tensor_by_name('input:0')
output_tensor = sess.graph.get_tensor_by_name('output/Softmax:0')
converter = tf.lite.TFLiteConverter.from_session(
    sess, [input_tensor], [output_tensor])
tflite = converter.convert()
open("auto_correction.tflite","wb").write(tflite)
print('Model converted successfully!')

Model converted successfully!


## Step 4: Check the converted TensorFlow Lite model.

We're just going to load the TensorFlow Lite model and use the TensorFlow Lite python interpreter to verify the results.

In [ ]:
# Step 4: Check the converted TensorFlow Lite model.
interpreter = tf.lite.Interpreter(model_content=tflite)

try:
  interpreter.allocate_tensors()
except ValueError:
  assert False

MINI_BATCH_SIZE = 1
correct_case = 0
for i in range(len(x_test)):
  input_index = (interpreter.get_input_details()[0]['index'])
  interpreter.set_tensor(input_index, x_test[i * MINI_BATCH_SIZE: (i + 1) * MINI_BATCH_SIZE])
  interpreter.invoke()
  output_index = (interpreter.get_output_details()[0]['index'])
  result = interpreter.get_tensor(output_index)
  # Reset all variables so it will not pollute other inferences.
  interpreter.reset_all_variables()
  # Evaluate.
  prediction = np.argmax(result)
  if prediction == y_test[i]:
    correct_case += 1

print('TensorFlow Lite Evaluation result is {}'.format(correct_case * 1.0 / len(x_test)))

In [31]:
tokenizer.json.d

AttributeError: 'Tokenizer' object has no attribute 'json'